In [1]:
import os
path = 'flower_images/'
names = sorted(os.listdir(path))
len(names)

211

In [2]:
import pandas as pd
df = pd.read_csv(os.path.join(path,names[-1]))
df.head()

,file,label
0,0001.png,0
1,0002.png,0
2,0003.png,2
3,0004.png,0
4,0005.png,0


In [0]:
dic = df.to_dict('index')
raw_data = []
for val in dic.values():
    raw_data.append(val)


In [0]:
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [0]:
data = []
for e in raw_data:
    temp = {}
    img_path = os.path.join(path,e['file'])
    img = image.load_img(img_path,target_size=(224,224))
    x = image.img_to_array(img)
    
    temp['array'] = x
    temp['label'] = e['label']
    data.append(temp)

In [0]:
X = []
y = []
for i in range(len(data)):
    X.append(data[i]['array'])

for j in range(len(data)):
    y.append(data[i]['label'])

X = np.array(X)
X = X.reshape(X.shape[0],224,224,3)
y = np.array(y)
y = y.reshape(y.shape[0],1)

In [7]:
final_data = np.array(list(zip(X,y)))
train_size = int(0.85*final_data.shape[0])
np.random.shuffle(final_data)

X_train = []
for i in range(train_size):
    X_train.append(final_data[:train_size,0][i])
X_train = np.array(X_train)

y_train = final_data[:train_size,1]
y_train = y_train.reshape(y_train.shape[0],1)

X_test = []
for j in range(final_data.shape[0] - train_size):
    X_test.append(final_data[train_size:,0][j])

X_test = np.array(X_test)
y_test = final_data[train_size:,1]
y_test = y_test.reshape(y_test.shape[0],1)
X_test.shape

(32, 224, 224, 3)

In [0]:
base_model = ResNet50(weights='imagenet',include_top=False)

In [0]:
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model

In [0]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
pred = Dense(10,activation='softmax')(x)

In [11]:
model = Model(inputs=base_model.input,outputs=pred)
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 conv1
2 bn_conv1
3 activation
4 max_pooling2d
5 res2a_branch2a
6 bn2a_branch2a
7 activation_1
8 res2a_branch2b
9 bn2a_branch2b
10 activation_2
11 res2a_branch2c
12 res2a_branch1
13 bn2a_branch2c
14 bn2a_branch1
15 add
16 activation_3
17 res2b_branch2a
18 bn2b_branch2a
19 activation_4
20 res2b_branch2b
21 bn2b_branch2b
22 activation_5
23 res2b_branch2c
24 bn2b_branch2c
25 add_1
26 activation_6
27 res2c_branch2a
28 bn2c_branch2a
29 activation_7
30 res2c_branch2b
31 bn2c_branch2b
32 activation_8
33 res2c_branch2c
34 bn2c_branch2c
35 add_2
36 activation_9
37 res3a_branch2a
38 bn3a_branch2a
39 activation_10
40 res3a_branch2b
41 bn3a_branch2b
42 activation_11
43 res3a_branch2c
44 res3a_branch1
45 bn3a_branch2c
46 bn3a_branch1
47 add_3
48 activation_12
49 res3b_branch2a
50 bn3b_branch2a
51 activation_13
52 res3b_branch2b
53 bn3b_branch2b
54 activation_14
55 res3b_branch2c
56 bn3b_branch2c
57 add_4
58 activation_15
59 res3c_branch2a
60 bn3c_branch2a
61 activation_16
62 res3c_branch

In [0]:
for layer in model.layers[:175]:
  layer.trainable = False

for layer in model.layers[175:]:
  layer.trainable = True

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
from tensorflow.python.keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes=10)
y_test = to_categorical(y_test,num_classes=10)

In [15]:
model.evaluate(X_test,y_test)

32/32 [==============================] - 2s 70ms/step


[2.2813310623168945, 0.09375]

In [16]:
hist = model.fit(X_train,y_train,epochs=10,batch_size=4)

Epoch 1/100
178/178 [==============================] - 4s 23ms/step - loss: 0.0434 - acc: 0.9831
Epoch 2/100
178/178 [==============================] - 3s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 3/100
178/178 [==============================] - 3s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 4/100
178/178 [==============================] - 3s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 5/100
178/178 [==============================] - 3s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 6/100
178/178 [==============================] - 3s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 7/100
178/178 [==============================] - 3s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 8/100
178/178 [==============================] - 3s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 9/100
178/178 [==============================] - 3s 15ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 10/100
178/178 [==============================] - 3s 15ms/step - loss: 1.1921e-07 - acc: 

In [17]:
model.evaluate(X_test,y_test)

32/32 [==============================] - 0s 10ms/step


[1.1920930376163597e-07, 1.0]

In [0]:
model.save('flower_model.h5')

In [0]:
from tensorflow.python.keras.models import load_model
m = load_model('flower_model.h5')

In [21]:
m.evaluate(X_test,y_test)

32/32 [==============================] - 1s 37ms/step


[1.1920930376163597e-07, 1.0]

In [0]:
base_model.save('ResNet-base.h5')

In [23]:
t = load_model('ResNet-base.h5')

In [24]:
t.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin